In [19]:
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
import numpy as np
import os
import pickle
import gc
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import Normalizer, StandardScaler
from utils import *

In [20]:
stock_symbol = '2454.TW'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
end_date = '2024-12-31'

# 擷取股票價格資訊
scaler = StandardScaler()
stock_price_data = fetch_stock_price(stock_symbol=stock_symbol, start_date='2012-01-02',end_date=end_date)

"""
pctchange: (today - yesterday)/yesterday
"""
stock_price_data['do'] = stock_price_data['Open'].pct_change() * 100
stock_price_data['dh'] = stock_price_data['High'].pct_change() * 100
stock_price_data['dl'] = stock_price_data['Low'].pct_change() * 100
stock_price_data['dc'] = stock_price_data['Close'].pct_change() * 100
stock_price_data['dv'] = stock_price_data['Volume'].pct_change() * 100

"""
do_1, dc_1, doc_1: tmr's information
"""
stock_price_data['do_1'] = stock_price_data['do'].shift(-1)
stock_price_data['dc_1'] = stock_price_data['dc'].shift(-1)
stock_price_data['doc_1'] = \
    ((stock_price_data['Close'].shift(-1) - stock_price_data['Open'].shift(-1))/stock_price_data['Open'].shift(-1))\
    *100

stock_price_data = stock_price_data.dropna()

# df = stock_price_data.iloc[:,7:]
df = stock_price_data
# Replace infinite values with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN values
df = df.dropna()

# scaling
df['Close_origin'] = df['Close']
scaler = StandardScaler()
scaler.fit(df[['do', 'dh', 'dl', 'dc', 'dv', 'Close']][:2000])
df[['do', 'dh', 'dl', 'dc', 'dv', 'Close']] = scaler.fit_transform(df[['do', 'dh', 'dl', 'dc', 'dv', 'Close']])


/home/jacob/anaconda3/envs/mlntu/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/tmp/ipykernel_515785/3269500910.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Close_origin'] = df['Close']
/tmp/ipykernel_515785/3269500910.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['do', 'dh', 'd

In [21]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,do,dh,dl,dc,dv,do_1,dc_1,doc_1,Close_origin
Date,,,,,,,,,,,,,,,,
2012-01-03 00:00:00+08:00,157.783940,158.915007,147.321564,-0.901834,16202000,0.0,0.0,-0.119655,0.140577,1.230761,1.121485,0.042897,1.075254,0.191924,-7.446809,147.321564
2012-01-04 00:00:00+08:00,159.480518,161.459886,147.604309,-0.900632,15873000,0.0,0.0,0.429574,0.778717,0.048725,0.044860,-0.039539,0.531922,8.045985,-0.529101,147.604309
2012-01-05 00:00:00+08:00,160.328830,160.328830,157.783928,-0.850137,13977000,0.0,0.0,0.191633,-0.401856,3.396716,3.617072,-0.054416,-0.705469,-1.241136,-1.065719,159.480530
2012-01-06 00:00:00+08:00,159.197761,159.197761,156.935626,-0.858553,10342000,0.0,0.0,-0.350258,-0.404395,-0.315584,-0.606930,-0.075517,-2.309064,-4.129269,-2.909091,157.501160
2012-01-09 00:00:00+08:00,155.521783,156.087317,150.714746,-0.886205,11573000,0.0,0.0,-1.052521,-1.044631,-2.026540,-1.920521,-0.018631,-2.363631,1.123601,0.558659,150.997513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-01 00:00:00+08:00,1165.000000,1165.000000,1095.000000,3.169979,12066326,0.0,0.0,1.922500,0.853245,-0.721917,-1.438820,0.011846,-4.291845,4.072398,3.139013,1105.000000
2024-03-04 00:00:00+08:00,1115.000000,1155.000000,1105.000000,3.361308,12304302,0.0,0.0,-1.920840,-0.482812,0.408919,1.809791,-0.033533,3.139013,0.434783,0.434783,1150.000000
2024-03-05 00:00:00+08:00,1150.000000,1160.000000,1130.000000,3.382567,8416339,0.0,0.0,1.333356,0.179432,1.082649,0.155317,-0.083907,-0.434783,3.030303,3.930131,1155.000000


In [22]:
num_class = 1
batch_size = 64

x, y, date = window_x_y(df, num_class, 10)
# X = process_x(x)
X, x_test, y, y_test = train_test(x, y)
x_train, x_valid, y_train, y_valid = train_valid(X, y)

trainloader, validloader, testloader = (
    loader(
        torch.tensor(x_train).to(dtype=torch.float32), 
        torch.tensor(y_train).to(dtype=torch.float32), 
        batch_size=batch_size), 
    loader(
        torch.tensor(x_valid).to(dtype=torch.float32), 
        torch.tensor(y_valid).to(dtype=torch.float32), 
        batch_size=batch_size),
    loader(
        torch.tensor(x_test).to(dtype=torch.float32), 
        torch.tensor(y_test).to(dtype=torch.float32), 
        batch_size=batch_size)
    )    

test_date = df.index[-len(y_test):]



  0%|          | 0/2953 [00:00<?, ?it/s]

100%|██████████| 2953/2953 [00:04<00:00, 719.40it/s]


In [23]:
len(x_train)

2125

### Double

In [24]:
def get_src():    
    x, y, date = window_x_y(df, num_class, 1)
    src = x[:2000]
    return torch.tensor(src).to(dtype=torch.float32)

src = get_src()
if num_class == 1:
    with open('./DataLoader/dataloader_1.pk', 'wb') as f:
        pickle.dump({'trainloader': trainloader, 'validloader': validloader, 'testloader': testloader}, f)
elif num_class == 2:
    with open('./DataLoader/dataloader.pk', 'wb') as f:
        pickle.dump({'trainloader': trainloader, 'validloader': validloader, 'testloader': testloader}, f)
with open('DataLoader/dates.pk', 'wb') as f:
    pickle.dump({'test': test_date}, f)
with open('DataLoader/data_clean.pk', 'wb') as f:
    pickle.dump(df, f)
with open('DataLoader/src.pk', 'wb') as f:
    pickle.dump(src, f)

100%|██████████| 2962/2962 [00:03<00:00, 900.76it/s] 


In [25]:
import pandas as pd
pd.DataFrame(y_test).set_index(df.index[-len(y_test):])

,0
Date,
2022-12-08 00:00:00+08:00,2.100840
2022-12-09 00:00:00+08:00,-0.419580
2022-12-12 00:00:00+08:00,-1.115760
2022-12-13 00:00:00+08:00,0.139470
2022-12-14 00:00:00+08:00,-3.129445
...,...
2024-03-01 00:00:00+08:00,3.139013
2024-03-04 00:00:00+08:00,0.434783
2024-03-05 00:00:00+08:00,3.930131


In [26]:

""" Standard
mean = torch.mean(x_train)
std = torch.std(x_train)
print(std)
x_train = normalize(x_train, mean, std)
mean = torch.mean(x_valid)
std = torch.std(x_valid)
print(std)
x_valid = normalize(x_valid, mean, std)
x_test = normalize(x_test, mean, std)
"""

"""src = use_src(df)
src = torch.tensor(src).to(dtype=torch.float32).unsqueeze(0).repeat(batch_size, 1, 1)
src.shape, x_train.shape"""

'src = use_src(df)\nsrc = torch.tensor(src).to(dtype=torch.float32).unsqueeze(0).repeat(batch_size, 1, 1)\nsrc.shape, x_train.shape'